In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import datetime as dt
import alpaca_trade_api as tradeapi
import json
from alpaca_trade_api.rest import REST, TimeFrame
from libs.drew_lib import *
load_dotenv()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

tradeapi = REST(alpaca_key, alpaca_secret_key)

In [ ]:
company_keywords = {
    'NFLX': ['NFLX', 'nflx', 'Netflix', 'netflix'],
    'FB': ['FB', 'fb', 'Facebook', 'facebook'],
    'UBER': ['UBER', 'uber', 'Uber'],
    'MCHP': ['MCHP', 'mchp', 'Microchip Technology'],
    'ABNB': ['ABNB', 'abnb', 'AirBnB', 'airbnb'],
    'FANG': ['FANG', 'fang', 'Diamondback Energy', 'diamondback energy', 'Diamondback', 'diamondback'],
    'MRO': ['MRO', 'mro', 'Marathon Oil', 'marathon oil'],
    'DVN': ['DVN', 'dvn', 'Devon Energy', 'devon energy'],
    'SPWR': ['SPWR', 'spwr', 'SunPower', 'Sunpower', 'sunpower'],
    'REGI': ['REGI', 'regi', 'Renewable Energy Group', 'renewable energy group'],
    'MTRX': ['MTRX', 'mtrx', 'McKinsey & Company', 'McKinsey & Co', 'Mckinsey & Co', 'McKinsey', 'Mckinsey', 'mckinsey'],
    'BLK': ['BLK', 'blk', 'BlackRock', 'Blackrock', 'blackrock'],
    'PYPL': ['PYPL', 'pypl', 'PayPal', 'Paypal', 'paypal'],
    'MELI': ['MELI', 'meli', 'MercadoLibre', 'Mercadolibre', 'mercadolibre'],
    'SOFI': ['SOFI', 'sofi', 'SoFi', 'Sofi']
}

In [3]:
# def get_daily_trade_data(ticker, start, end, tradeapi): 
#     ticker_df = tradeapi.get_bars(
#         ticker,
#         TimeFrame.Day,
#         start,
#         end
#     ).df
#     ticker_df['ticker'] = ticker

#     return ticker_df

In [4]:
# ticker_list = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

# def make_tickers_df(ticker_list, start_date_str, end_date_str, tradeapi):
#     ticker_dfs_list = [get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list]
#     tickers_df = pd.concat(ticker_dfs_list, axis=0, join='outer')
#     tickers_df.index = tickers_df.index.date
#     tickers_df = tickers_df[['ticker','close','volume']]
#     return tickers_df


In [5]:
# def make_tickers_dict(ticker_list, start_date_str, end_date_str, tradeapi):
#     return {ticker: get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list}

## News Article Vader Analyzer

In [6]:
from libs.drew_lib import *

from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
api_key = os.getenv("NEWSAPI_KEY")
newsapi = NewsApiClient(api_key=api_key)
print(type(api_key))

<class 'str'>


In [8]:
# company_df = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')
# company_df = company_df.rename(columns={'Company Name': 'Company_Name'})
# company_dict = dict(zip(company_df.Company_Name, company_df.Ticker))
# company_list = list(company_dict.keys())
# tickers_list = company_dict.values()
# ls = [type(item) for item in company_list]
# print(ls)
# tickers_list


In [9]:
def vader_analyzer(df):
    analyzer = SentimentIntensityAnalyzer()
    df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]
    df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['text']]
    df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['text']]
    df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['text']]

    df['date'] = pd.to_datetime(
    df['date'],
    infer_datetime_format = True,
    utc = True    
    )
    df['date'] = df['date'].dt.date
    
    return df



In [18]:
news_df_list = [form_df(company) for company in company_list]
news_df = pd.concat(news_df_list, axis=0, join='outer')
vader_news_df = vader_analyzer(news_df)
news_sent_avg = vader_news_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
news_sent_avg = news_sent_avg[['date','ticker','pos','neg','neu','compound']]
news_sent_avg

#news_sent_avg.to_csv('../Notebooks/Data/Cleaned_Data/news_sentiment.csv')

C:\Users\altma\AppData\Local\Temp\ipykernel_42908\2360381749.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  news_sent_avg = vader_news_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()


,date,ticker,pos,neg,neu,compound
0,2022-05-11,ABNB,0.075167,0.0260,0.898667,0.199033
1,2022-05-12,ABNB,0.210000,0.1590,0.631000,0.491200
2,2022-05-14,ABNB,0.169000,0.0000,0.831000,0.709950
3,2022-05-16,ABNB,0.000000,0.0000,1.000000,0.000000
4,2022-05-18,ABNB,0.000000,0.0480,0.952000,-0.278700
...,...,...,...,...,...,...
182,2022-05-20,UBER,0.000000,0.1770,0.823000,-0.759900
183,2022-05-24,UBER,0.000000,0.0660,0.934000,-0.296000
184,2022-05-25,UBER,0.000000,0.0400,0.960000,-0.128000
185,2022-06-06,UBER,0.053000,0.0485,0.898000,0.159100


In [17]:
news_df_list = [form_df(company) for company in company_list]
vader_news_df_list = [daily_sentiment(df) for df in news_df_list]
vader_news_df = pd.concat(vader_news_df_list, axis=0, join='outer')
vader_news_df.head()

C:\Users\altma\AppData\Local\Temp\ipykernel_42908\606697139.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vader_df = vader_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
C:\Users\altma\AppData\Local\Temp\ipykernel_42908\606697139.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vader_df = vader_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
C:\Users\altma\AppData\Local\Temp\ipykernel_42908\606697139.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vader_df = vader_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
C:\Users\altma\AppData\Local\Temp\ipykernel_42908\606697139.py:5: FutureWarning: Indexing with multi

,date,ticker,pos,neg,neu,compound
0,2022-05-08,NFLX,0.064000,0.000000,0.936000,0.340000
1,2022-05-09,NFLX,0.000000,0.000000,1.000000,0.000000
2,2022-05-10,NFLX,0.074333,0.044667,0.881333,0.189467
3,2022-05-13,NFLX,0.110000,0.000000,0.890000,0.475300
4,2022-05-14,NFLX,0.077000,0.000000,0.923000,0.361200


In [11]:
# perform sentiment analysis and find average sentiment by date

def daily_sentiment(df):
    vader_df = vader_analyzer(df)
    vader_df = vader_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
    vader_df = vader_df[['date','ticker','pos','neg','neu','compound']]
    return vader_df

## Helper Functions for PuttingItTogether

In [13]:
def return_bin(df):

    bins = [-1000, -0.08, -0.03, -0.0000001, 0.0000001, 0.03, 0.08, 1000]
    labels = ['large loss','medium loss','small loss','no gain/loss','small gain','medium gain','large gain']
    
    df['returns'] = df['close'].pct_change()
    df['return_bin'] = pd.cut(df['returns'], bins=bins, labels=labels)

    return df

## Data prep for model

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [15]:
nflx_df = stock_picker('NFLX')
nflx_df = return_bin(nflx_df)
nflx_df = nflx_df[nflx_df['return_bin'].notna()]
nflx_df.head()

ValueError: 'date' is not in list

In [ ]:
X = nflx_df.copy()
X = nflx_df.drop(columns=['return_bin','ticker'])
y = nflx_df['return_bin'].values
y = y.reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
X_scaler = StandardScaler()
X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
enc = OneHotEncoder()
enc.fit(y_train)

y_train_enc = enc.transform(y_train).toarray()
y_test_enc = enc.transform(y_test).toarray()

In [ ]:
model = Sequential()

number_units = 30
dropout_fraction = 0.2

In [ ]:
# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train_scaled.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model
model.fit(X_train_scaled, y_train_enc, epochs=20, shuffle=False, batch_size=90, verbose=1)

# Evaluate the model
model.evaluate(X_test_scaled, y_test_enc, verbose=0)

# Make predictions using the testing data X_test
predicted = model.predict(X_test_scaled)

Epoch 1/20
14/14 [==============================] - 3s 7ms/step - loss: nan
Epoch 2/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 3/20
14/14 [==============================] - 0s 8ms/step - loss: nan
Epoch 4/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 5/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 6/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 7/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 8/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 9/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 10/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 11/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 12/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 13/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 14

In [ ]:
predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test.activity.values,
    "Predicted": predicted
})
results.head(10)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').